In [1]:
using StateSpaceDynamics
using Random
using Distributions
using Plots
using LinearAlgebra
using ForwardDiff
using StatsFuns
using Optim

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [2]:
# generate synthetic data for a switching Gaussian Regression
Random.seed!(12345)

n = 1000

beta_1 = [1.0, 2.0]
beta_2 = [3.0, 4.0]

sigma_1 = 0.5
sigma_2 = 0.5

# Simualte data from a switching GLM
A = [0.9 0.1; 0.1 0.9]
pi_k = [0.5, 0.5]

X = randn(n, 2)
Y = zeros(n)
states = zeros(Int, n)

# First state
states[1] = Int(rand() < pi_k[1] ? 1 : 2)
Y[1] = if states[1] == 1
    beta_1[1] + beta_1[2] * X[1, 1] + sigma_1 * randn()
else
    beta_2[1] + beta_2[2] * X[1, 1] + sigma_2 * randn()
end

# Remaining states
for i in 2:n
    states[i] = Int(rand() < A[states[i-1], 1] ? 1 : 2)
    Y[i] = if states[i] == 1
        beta_1[1] + beta_1[2] * X[i, 1] + sigma_1 * randn()
    else
        beta_2[1] + beta_2[2] * X[i, 1] + sigma_2 * randn()
    end
end


In [3]:
# reshape x and y to be matrices
X_input = reshape(X[:, 1], n, 1)
Y = reshape(Y, n, 1)

est_model = SwitchingGaussianRegression(K=2, input_dim=1, output_dim=1)
fit!(est_model, X_input, Y)

New Function in use...


Running EM algorithm...   2%|█                           |  ETA: 0:08:04

Running EM algorithm...  15%|█████                       |  ETA: 0:01:10

Running EM algorithm...  27%|████████                    |  ETA: 0:00:37

Running EM algorithm...  37%|███████████                 |  ETA: 0:00:25

Running EM algorithm...  47%|██████████████              |  ETA: 0:00:18

Running EM algorithm...  56%|████████████████            |  ETA: 0:00:13

Running EM algorithm...  66%|███████████████████         |  ETA: 0:00:09

Running EM algorithm...  77%|██████████████████████      |  ETA: 0:00:06

Running EM algorithm...  87%|█████████████████████████   |  ETA: 0:00:03

Running EM algorithm...  97%|████████████████████████████|  ETA: 0:00:01

Running EM algorithm... 100%|████████████████████████████| Time: 0:00:21
  iteration:       100
  log_likelihood:  -74841.85316699692


101-element Vector{Float64}:
     -Inf
   -1433.172649791336
 -545879.6836785846
   -6396.079079617949
  -14008.250675089454
  -34858.99998635821
  -89999.15109876319
 -201737.7469911903
  -74085.39317889638
 -274140.78731034446
       ⋮
 -249344.04075633225
 -249768.33771563595
 -249822.97676943138
  -74832.55621760219
  -74891.69067695455
  -74903.57130075173
 -249346.75341842565
  -74841.85316699692
 -249421.75666097453

In [4]:
est_model

HiddenMarkovModel([0.7532219087358173 0.24677809126416506; 0.3177473678839959 0.6822526321159564], EmissionModel[GaussianRegressionEmission(1, 1, [-0.7266347042654221; 0.24502849424517972;;], [0.00020071105963722431;;], true, 0.0), GaussianRegressionEmission(1, 1, [-0.3998203261011363; 0.4639521824879004;;], [0.00020071105963722431;;], true, 0.0)], [0.0, 1.0], 2)

In [4]:
est_model

HiddenMarkovModel([0.9038925880407833 0.09610741195921672; 0.9332951140632878 0.06670488593671217], EmissionModel[GaussianRegressionEmission(1, 1, [0.0; 0.0;;], [1.0;;], true, 0.0), GaussianRegressionEmission(1, 1, [0.0; 0.0;;], [1.0;;], true, 0.0)], [0.627130237478469, 0.372869762521531], 2)

In [ ]:
# generate synthetic data
Random.seed!(123)

# set parameters
n = 1000
β = [0.1, 0.9]

# generate data
X = hcat(ones(n), randn(n, 1))

p = logistic.(X * β)
y = rand.(Bernoulli.(p));

logreg = SSM.BernoulliRegression()
SSM.fit!(logreg, reshape(X[:, 2], length(y), 1), y)

In [ ]:
model = SSM.SwitchingBernoulliRegression(;K=2, λ=0.1)
SSM.fit!(model, reshape(X[:, 2], length(y), 1), y, 10)

In [ ]:
# synthetic poisson regression data
Random.seed!(123)

# set parameters
n = 100
β = [0.1, 0.9]
# generate data

X = hcat(ones(n), randn(n, 1))
λ = exp.(X * β)

y = rand.(Poisson.(λ))

poisreg = SSM.PoissonRegression()
SSM.fit!(poisreg, reshape(X[:, 2], length(y), 1), y)

In [ ]:
model = SSM.SwitchingPoissonRegression(;K=2, λ=0.1)
fit!(model, reshape(X[:, 2], length(y), 1), y)

In [ ]:
# Parameters
n = 1000
β₁ = [0.1, 0.9]
β₂ = [0.9, 0.1]
σ²₁ = 0.1
σ²₂ = 0.1
A = [0.9 0.1; 0.1 0.9]

# Data generation
X = hcat(ones(n), randn(n, 1))
y = zeros(n)
state = Vector{Int}(undef, n)

for i in 1:n
    if i == 1
        # choose a state at random
        state[i] = rand(1:2)
    else
        # Transition based on previous state
        state[i] = rand(Categorical(A[state[i-1], :]))
    end

    if state[i] == 1
        y[i] = X[i, :]' * β₁ + sqrt(σ²₁) * randn()
    else
        y[i] = X[i, :]' * β₂ + sqrt(σ²₂) * randn()
    end
end

# Fit linear regression model
linreg = SSM.SwitchingGaussianRegression(; K=2, λ=0.1)
SSM.fit!(linreg, reshape(X[:, 2], length(y), 1), y, 10000, 1e-6)

println("Fitted coefficients: ", linreg.B[1].regression.β)


In [ ]:
γ, ξ, α, β = SSM.E_step(linreg, reshape(X[:, 2], length(y), 1), y)